In [1]:
import pandas as pd
import numpy as np
import pulp
import time

In [2]:
rng = np.random.RandomState(4)

# Número de mochilas
n = 40 
# Número de itens
m = 80
# Duracao de um item
d = rng.randint(1, 9, m)
# Tempo da viagem
t = rng.randint(24, 241, n)
# Valor de um item
c = rng.randint(50, 2001, m)
# Orçamento
b = rng.randint(2000, 10001, n)
# Estoque de cada elemento
e = rng.randint(1, int(n + 1), m)
# Avaliacao
w = rng.randint(-10, 11, (m, n))

In [10]:
# Construir a tabela de benefícios
def benefitTable() :
    index = ['Item ' + str(i) for i in range(m)]
    columns = ['Mochila ' + str(k) for k in range(n)]
    table = pd.DataFrame(np.array(w), index = index, columns = columns)
    print table.to_latex()
    return display(table)

In [4]:
# Construir a tabela de restrições das mochilas
def knapsacksRestritions():
    index_bt = ['Custo ' , 'Tempo']
    columns_bt = ['Mochila ' + str(k) for k in range(n)]
    table_bt = pd.DataFrame(np.array([b, t]), index = index_bt, columns = columns_bt)
    return display(table_bt)

In [5]:
# Construir a tabela de características dos itens
def itemsFeatures() :
    index_cde = ['Custo ' , 'Duração', 'Estoque']
    columns_cde = ['Item ' + str(i) for i in range(m)]
    table_cde = pd.DataFrame(np.array([c, d, e]), index = index_cde, columns = columns_cde)
    return display(table_cde)

In [14]:
def alg1() :
    # ALGORITMO 1 (VIZINHO MAIS PRóX - MOCHILA ALEATÓRIA E TENDO ADD OS MELHORES ITENS NELA)
    library = np.array(e)
    z = 0
    for k in range(n): # Para cada mochila
        # Reconstruir a tabela de benefícios por mochila, ordenando-os
        index_1 = ['i_' + str(i) for i in range(m)]
        columns_1 = ['k_' + str(k)]
        matrix_1 = np.array(w)[:, k]
        table_1 = pd.DataFrame(matrix_1, index = index_1, columns = columns_1)
        table_1['avg'] = [table_1.iloc[i,0]/c[i]/d[i] for i in range(m)]
        table_1 = table_1.sort_values(by=['avg'], ascending=False)
        #display(table_1)
        #print ('k_' + str(k) + ', capacidade de custo ' + str(b[k]) + ', capacidade de tempo ' + str(t[k]) )
        #print (' ')
        budget = 0 #capacidade atual de custo
        duration = 0 #capacidade atual de tempo
        solution = []
        sum_solution = 0
        for i in range(table_1.shape[0]): #Para cada item na tabela
            weight = table_1.iloc[i, 0]
            item_no = int(table_1.index[i][2:])
            inventory = library[item_no]
            cost = c[item_no]
            time = d[item_no]
            if inventory > 0 and budget + cost <= b[k] and duration + time <= t[k] and sum_solution + weight >= sum_solution: # a adição deste item não piorar o conjunto solução, então:
                library[item_no] += -1 # remove um item do estoque
                budget += cost # adiciona o custo do item ao budget
                duration += time # adiciona o tempo do item a duração da viagem
                solution.append(item_no) # adiciona o item ao conjunto solução
                sum_solution += weight # adiciona o peso do item ao conjunto solução
                # print ('\titem ' + str(item_no) + '*, estoque ' + str(inventory) + ', custo ' + str(cost) + ', duração ' + str(time) + ', benefício ' + str(weight))
            #else:
                #print ('\titem ' + str(item_no) + ', estoque ' + str(inventory) + ', custo ' + str(cost) + ', duração ' + str(time) + ', benefício ' + str(weight))        
        #print (' ')
        #print ('Mochila ' + str(k) + ' -> solução: ' + str(solution) + ' -> benefício: ' + str(sum_solution))
        z += sum_solution
    #print ('Z= ' + str(z))
    #print(' ')
    return z

In [15]:
def alg2():
    # ALGORITMO 2 (INSERÇAO DO MAIS BARATO - ITEM ALEATÓRIO E TENDO ADD ELE NAS MOCHILAS QUE TEM O MAIOR VALOR)
    z = 0
    # Reconstruir a tabela de benefícios por mochila, ordenando-os
    index_2 = ['k_' + str(k) for k in range(n)]
    columns_2 = ['i_' + str(i) for i in range(m)]
    matrix_2 = np.array(w).transpose()
    table_2 = pd.DataFrame(matrix_2, index = index_2, columns = columns_2)
    table_2['avg'] = table_2.mean(axis=1)
    table_2 = table_2.sort_values(by=['avg'], ascending=False)
    #display(table_2)

    budget = {}
    duration = {}
    solution ={}
    sum_solution={}
    for k in range(n): # Para cada mochila
        budget["k_{0}".format(k)] = 0 #capacidade atual de custo
        duration["k_{0}".format(k)] = 0 #capacidade atual de tempo
        solution["k_{0}".format(k)] = []
        sum_solution["k_{0}".format(k)] = 0
    #print('budget: ' + str(budget)) 
    #print('duration: ' + str(duration))
    #print('solution: ' + str(solution))
    #print('sum_solution: ' + str(sum_solution))
    #print(' ')

    for i in range(m): #Para cada item na tabela
        item_no = int(table_2.columns[i][2:])
        inventory = np.array(e)[item_no]
        cost = c[item_no]
        time = d[item_no]
        #print ('item: ' + str(item_no) + ', cost ' + str(cost) + ', time ' + str(time) +', inventory ' + str(inventory))
        table_2 = table_2.sort_values(by=['i_'+str(item_no)], ascending=False)
        #display(table_2)
        k_loop = 0
        while inventory != 0 and k_loop < n:
            #print(str(k_loop+1)+ '/' + str(n))
            weight = table_2.iloc[k_loop,i]
            knap = int(table_2.index[k_loop][2:])
            #print('weight is: ' + str(weight) + ' in knapsack: '+str(knap))
            the_k = 'k_' + str(knap)
            if inventory > 0 and budget[the_k] + cost <= b[knap] and duration[the_k] + time <= t[knap] and sum_solution[the_k] + weight >= sum_solution[the_k]: # a adição deste item não piorar o conjunto solução, então:
                inventory += -1 # remove um item do estoque
                budget[the_k] += cost # adiciona o custo do item ao budget
                duration[the_k] += time # adiciona o tempo do item a duração da viagem
                solution[the_k].append(item_no) # adiciona o item ao conjunto solução
                sum_solution[the_k] += weight # adiciona o peso do item ao conjunto solução
                #print ('item ' + str(item_no) + ' -> mochila ' + str(the_k) +', estoque ' + str(inventory) + ', custo ' + str(cost) + ', duração ' + str(time) + ', benefício ' + str(weight))
                #print (' ')
            #else: 
                #print ('cant insert. Try to add to next knap')
                #print (' ')
            k_loop += 1

    for k in range(n): # Para cada mochila
        the_k = 'k_' + str(k)
        #print ('Mochila ' + str(k) + ' -> solução: ' + str(solution[the_k]) + ' -> benefício: ' + str(sum_solution[the_k]))
        z += sum_solution[the_k]
    #print ('Z= ' + str(z))
    #print(' ')
    return z

In [16]:
def get_max_index(df):
    col = df.max(axis = 0, skipna=True).sort_values(ascending = False).index[0]
    index = df[col].sort_values(ascending = False).index[0]
    return (index, col)

In [17]:
def alg3():
    # ALGORITMO 3 (INSERÇAO DO MAIS PROXIMO - PEGA O MELHOR ITEM DE TODOS E ADD NA MELHOR MOCHILA)
    z = 0
    # Reconstruir a tabela de benefícios por mochila, ordenando-os
    index_3 = ['i_' + str(i) for i in range(m)]
    columns_3 = ['k_' + str(k) for k in range(n)]
    matrix_3 = np.array(w)
    table_3 = pd.DataFrame(matrix_3, index = index_3, columns = columns_3)
    #display(table_3)

    budget = {}
    duration = {}
    solution ={}
    sum_solution={}
    for k in range(n): # Para cada mochila
        knap_no = k
        budget["k_{0}".format(k)] = 0 #capacidade atual de custo
        duration["k_{0}".format(k)] = 0 #capacidade atual de tempo
        solution["k_{0}".format(k)] = []
        sum_solution["k_{0}".format(k)] = 0
    #print('budget: ' + str(budget)) 
    #print('duration: ' + str(duration))
    #print('solution: ' + str(solution))
    #print('sum_solution: ' + str(sum_solution))
    #print(' ')

    interations = m * n

    library = np.array(e)

    while interations > 0:
        tpl = get_max_index(table_3)
        item = int(tpl[0][2:])
        knap = int(tpl[1][2:])
        the_k = tpl[1]
        inventory = library[item]
        cost = c[item]
        time = d[item]
        weight = table_3.iloc[item, knap]
        #print ('item: ' + str(item) + ', cost ' + str(cost) + ', time ' + str(time) +', inventory ' + str(inventory) + ', weight in k ' + str(weight))

        if inventory > 0 and budget[the_k] + cost <= b[knap] and duration[the_k] + time <= t[knap] and sum_solution[the_k] + weight >= sum_solution[the_k]: # a adição deste item não piorar o conjunto solução, então:
            library[item] += -1 # remove um item do estoque
            budget[the_k] += cost # adiciona o custo do item ao budget
            duration[the_k] += time # adiciona o tempo do item a duração da viagem
            solution[the_k].append(item) # adiciona o item ao conjunto solução
            sum_solution[the_k] += weight # adiciona o peso do item ao conjunto solução
            #print ('item ' + str(item) + ' -> mochila ' + str(the_k) +', estoque ' + str(inventory) + ', custo ' + str(cost) + ', duração ' + str(time) + ', benefício ' + str(weight))
            table_3.iloc[item, knap] = None
            #display(table_3)
        else: 
            table_3.iloc[item, knap] = None
            #display(table_3)

        interations += -1

    for k in range(n): # Para cada mochila
        the_k = 'k_' + str(k)
        #print ('Mochila ' + str(k) + ' -> solução: ' + str(solution[the_k]) + ' -> benefício: ' + str(sum_solution[the_k]))
        z += sum_solution[the_k]
    #print ('Z= ' + str(z))
    #print(' ')
    return z

In [18]:
def get_min_index(df):
    col = df.min(axis = 0, skipna=True).sort_values(ascending = True).index[0]
    index = df[col].sort_values(ascending = True).index[0]
    return (index, col)

In [19]:
def alg4():
    # ALGORITMO 4 (INSERÇAO DO MAIS DISTANTE - PEGA O PIOR ITEM DE TODOS E ADD NA MELHOR MOCHILA)
    z = 0
    # Reconstruir a tabela de benefícios por mochila, ordenando-os
    index_4 = ['i_' + str(i) for i in range(m)]
    columns_4 = ['k_' + str(k) for k in range(n)]
    matrix_4 = np.array(w)
    table_4 = pd.DataFrame(matrix_4, index = index_4, columns = columns_4)
    #display(table_4)

    budget = {}
    duration = {}
    solution ={}
    sum_solution={}
    for k in range(n): # Para cada mochila
        knap_no = k
        budget["k_{0}".format(k)] = 0 #capacidade atual de custo
        duration["k_{0}".format(k)] = 0 #capacidade atual de tempo
        solution["k_{0}".format(k)] = []
        sum_solution["k_{0}".format(k)] = 0
    #print('budget: ' + str(budget)) 
    #print('duration: ' + str(duration))
    #print('solution: ' + str(solution))
    #print('sum_solution: ' + str(sum_solution))
    #print(' ')

    interations = m * n
    library = np.array(e)

    iteration_list = table_4
    iteration_result_list = table_4

    while interations > 0:
        tpl = get_min_index(iteration_list)
        item = int(tpl[0][2:])
        old_knap = int(tpl[1][2:])
        inventory = library[item]
        cost = c[item]
        time = d[item]
        the_k = table_4.idxmax(1)[item]
        knap = int(the_k[2:])
        weight = table_4.iloc[item, knap]
    #    print ('item: ' + str(item) + ', cost ' + str(cost) + ', time ' + str(time) +', inventory ' + str(inventory) + ', weight in k ' + str(weight))

        if inventory > 0 and budget[the_k] + cost <= b[knap] and duration[the_k] + time <= t[knap] and sum_solution[the_k] + weight >= sum_solution[the_k]: # a adição deste item não piorar o conjunto solução, então:
            library[item] += -1 # remove um item do estoque
            budget[the_k] += cost # adiciona o custo do item ao budget
            duration[the_k] += time # adiciona o tempo do item a duração da viagem
            solution[the_k].append(item) # adiciona o item ao conjunto solução
            sum_solution[the_k] += weight # adiciona o peso do item ao conjunto solução
            #print ('item ' + str(item) + ' -> mochila ' + str(the_k) +', estoque ' + str(inventory) + ', custo ' + str(cost) + ', duração ' + str(time) + ', benefício ' + str(weight))
            iteration_list.iloc[item, old_knap] = None
            #display(iteration_list)
        else: 
            iteration_list.iloc[item, old_knap] = None
            #display(iteration_list)
        interations += -1

    for k in range(n): # Para cada mochila
        the_k = 'k_' + str(k)
        #print ('Mochila ' + str(k) + ' -> solução: ' + str(solution[the_k]) + ' -> benefício: ' + str(sum_solution[the_k]))
        z += sum_solution[the_k]
    #print ('Z= ' + str(z))
    #print(' ')
    return z

In [20]:
def algOptimal():
    #Definir o problema como minimização de uma função # LpMaximize ou LpMinimize
    simplex = pulp.LpProblem('Simplex', pulp.LpMaximize) 
    x_var = np.array([[pulp.LpVariable("x_{}_{}".format(i, k), cat='Binary') for k in range(n)] for i in range(m)])
    w_array = np.array(w)

    #Definir função objetivo
    simplex += sum(x_var[i, k] * w_array[i, k] for k in range(n) for i in range(m))

    #Definir restrições
    for k in range (n): #restrições da mochila
        simplex += (sum(x_var[i, k] * d[i] for i in range(m)) <= t[k])
        simplex += (sum(x_var[i, k] * c[i] for i in range(m)) <= b[k])
    for i in range (m) : #restrições de itens
        simplex += (sum(x_var[i, k] for k in range(n)) <= e[i])

    #print(simplex)
    simplex.solve()
    #print(pulp.LpStatus[simplex.status])


    #for variable in simplex.variables():
        #print("{} = {}".format(variable.name, variable.varValue))

    return pulp.value(simplex.objective)

In [21]:
#benefitTable()
#knapsacksRestritions()
#itemsFeatures()
alg1_start = time.time()
response = alg1()
alg1_finish = time.time()
print('alg1', response, alg1_finish - alg1_start)

alg2_start = time.time()
response = alg2()
alg2_finish = time.time()
print('alg2', response, alg2_finish - alg2_start)

alg3_start = time.time()
response = alg3()
alg3_finish = time.time()
print('alg3', response, alg3_finish - alg3_start)

alg4_start = time.time()
response = alg4()
alg4_finish = time.time()
print('alg4', response, alg4_finish - alg4_start)

algOptimal_start = time.time()
response = algOptimal()
algOptimal_finish = time.time()
print('algOptimal', response, algOptimal_finish - algOptimal_start)
print('total', algOptimal_finish - alg1_start)

('alg1', 1621, 0.16175103187561035)
('alg2', 1495, 0.08634805679321289)
('alg3', 2414.0, 5.45806097984314)
('alg4', 2540.0, 7.306588888168335)
('algOptimal', 3015.0, 437.69229912757874)
('total', 450.7061810493469)


In [11]:
benefitTable()
knapsacksRestritions()
itemsFeatures()

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr}
\toprule
{} &  Mochila 0 &  Mochila 1 &  Mochila 2 &  Mochila 3 &  Mochila 4 &  Mochila 5 &  Mochila 6 &  Mochila 7 &  Mochila 8 &  Mochila 9 &  Mochila 10 &  Mochila 11 &  Mochila 12 &  Mochila 13 &  Mochila 14 &  Mochila 15 &  Mochila 16 &  Mochila 17 &  Mochila 18 &  Mochila 19 &  Mochila 20 &  Mochila 21 &  Mochila 22 &  Mochila 23 &  Mochila 24 &  Mochila 25 &  Mochila 26 &  Mochila 27 &  Mochila 28 &  Mochila 29 &  Mochila 30 &  Mochila 31 &  Mochila 32 &  Mochila 33 &  Mochila 34 &  Mochila 35 &  Mochila 36 &  Mochila 37 &  Mochila 38 &  Mochila 39 \\
\midrule
Item 0  &          5 &          4 &         -7 &         -3 &         -8 &         -3 &         -2 &          1 &         -1 &          4 &           7 &          -8 &          -1 &           6 &          -5 &          -2 &          -9 &          -8 &          -7 &           6 &          -4 &           3 &         -10 &          -5 &           9 &           0 &    

,Mochila 0,Mochila 1,Mochila 2,Mochila 3,Mochila 4,Mochila 5,Mochila 6,Mochila 7,Mochila 8,Mochila 9,...,Mochila 30,Mochila 31,Mochila 32,Mochila 33,Mochila 34,Mochila 35,Mochila 36,Mochila 37,Mochila 38,Mochila 39
Item 0,5,4,-7,-3,-8,-3,-2,1,-1,4,...,-1,0,-2,7,5,-1,-9,-10,1,6
Item 1,6,8,5,2,-1,6,8,-1,0,-9,...,-3,-4,2,-8,-2,10,-8,-4,7,6
Item 2,10,-10,-3,6,8,9,3,6,-10,-9,...,-7,1,-9,9,-5,-9,10,-5,3,-9
Item 3,4,6,-5,9,7,-2,-10,3,1,-1,...,-4,3,-3,-4,-1,-5,-8,3,4,4
Item 4,-6,-3,-5,9,8,3,4,-8,-6,-1,...,5,6,-2,4,-9,-8,10,6,4,-3
Item 5,5,-10,2,4,7,-1,1,-7,-10,8,...,-3,5,-4,0,1,6,-3,-7,10,-10
Item 6,4,0,0,3,5,6,7,-10,4,-6,...,7,-1,1,-1,10,8,9,-3,3,-4
Item 7,0,-1,8,4,4,0,-9,1,5,-7,...,-10,-2,10,-3,6,3,-4,-3,10,-7
Item 8,-5,2,8,4,-2,7,-4,-3,-7,-3,...,-7,6,8,4,-8,-1,8,-5,-2,1
Item 9,-3,-6,-9,-3,-6,-5,-1,-3,-7,-5,...,-1,3,5,-4,8,-2,-3,-2,2,-1


,Mochila 0,Mochila 1,Mochila 2,Mochila 3,Mochila 4,Mochila 5,Mochila 6,Mochila 7,Mochila 8,Mochila 9,...,Mochila 30,Mochila 31,Mochila 32,Mochila 33,Mochila 34,Mochila 35,Mochila 36,Mochila 37,Mochila 38,Mochila 39
Custo,7016,6594,9197,4013,6644,7565,3026,8035,8864,3251,...,6825,2589,9985,7636,4408,3795,6492,9136,6850,6596
Tempo,47,80,138,222,223,225,234,207,205,109,...,102,33,149,77,220,214,138,211,155,28


,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 70,Item 71,Item 72,Item 73,Item 74,Item 75,Item 76,Item 77,Item 78,Item 79
Custo,985,1128,602,1326,1810,629,1565,1767,1736,798,...,1701,1522,471,1106,1355,1879,1561,1079,1531,892
Duração,3,7,8,6,2,1,8,1,3,2,...,1,5,8,3,7,7,4,7,7,4
Estoque,27,7,18,36,18,16,16,36,35,29,...,28,26,4,16,29,17,10,24,23,40
